# Notebook Under Construction

This tutorial showed how to train a model for image classification, test it, convert it to the TensorFlow Lite format for on-device applications (such as an image classification app), and perform inference with the TensorFlow Lite model with the Python API.

1. Setup
1. Load the trained moddel
1. Predict on sample data
1. Compress model for mobile, embeddded or edge device
1. Run TF Lite model
1. Predct with compressed model

# Setup

In [ ]:
!pip install -U pip --quiet
!pip install -r ../../requirements.txt --quiet

import matplotlib.pyplot as plt
import numpy as np
import PIL

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model, load_model
from tensorflow.keras.layers import Dense, Dropout,Activation, Flatten, Conv2D, MaxPool2D
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print("tensorflow version:  " + tf.__version__)

In [ ]:
# this directory is apart of the .gitignore to ensure it is not committed to git
%env SCRATCH=../scratch
![ -e "${SCRATCH}" ] || mkdir -p "${SCRATCH}"/model

import os
scratch_path = os.environ.get('SCRATCH', '../scratch')

# Load the trained model

In [ ]:
model = keras.models.load_model( scratch_path + '/model/left-right-print-prediction-1.0.0.h5')

# Compress model for mobile, embedded or edge device with TensorFlow Lite

TensorFlow Lite is a set of tools that enables on-device machine learning by helping developers run their models on mobile, embedded, and edge devices.

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open(scratch_path + '/model.tflite', 'wb') as f:
  f.write(tflite_model)

# Run the TF Lite model

You can access the TensorFlow Lite saved model signatures in Python via the tf.lite.Interpreter class.

In [ ]:
TF_MODEL_FILE_PATH = scratch_path + '/model.tflite' # The default path to the saved TensorFlow Lite model

interpreter = tf.lite.Interpreter(model_path=TF_MODEL_FILE_PATH)

Print the signatures from the converted model to obtain the names of the inputs (and outputs):

In [ ]:
interpreter.get_signature_list()

In this example, you have one default signature called serving_default. In addition, the name of the 'inputs' is 'sequential_1_input', while the 'outputs' are called 'outputs'. You can look up these first and last Keras layer names when running Model.summary, as demonstrated earlier in this tutorial.

Now you can test the loaded TensorFlow Model by performing inference on a sample image with tf.lite.Interpreter.get_signature_runner by passing the signature name as follows:

In [ ]:
classify_lite = interpreter.get_signature_runner('serving_default')
classify_lite

Similar to what you did earlier in the tutorial, you can use the TensorFlow Lite model to classify images that weren't included in the training or validation sets.

You have already tensorized that image and saved it as img_array. Now, pass it to the first argument (the name of the 'inputs') of the loaded TensorFlow Lite model (predictions_lite), compute softmax activations, and then print the prediction for the class with the highest computed probability.

In [ ]:
predictions_lite = classify_lite(sequential_1=img_array)['outputs']
score_lite = tf.nn.softmax(predictions_lite)

In [ ]:
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score_lite)], 100 * np.max(score_lite))
)

The prediction generated by the lite model should be almost identical to the predictions generated by the original model:

In [ ]:
print(np.max(np.abs(predictions - predictions_lite)))